In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import torch
from torchvision import datasets, transforms, models  # datsets  , transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
import torchsummary

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
transform = transforms.Compose(
    [transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor()]
)



dataset = datasets.ImageFolder(r"C:\Dataset", transform=transform)


In [ ]:
dataset = datasets.ImageFolder("C:\Dataset", transform=transform)

In [ ]:
dataset

In [ ]:
indices = list(range(len(dataset)))

In [ ]:
split = int(np.floor(0.85 * len(dataset)))

In [ ]:
validation = int(np.floor(0.70 * split))

In [ ]:
print(0, validation, split, len(dataset))

In [ ]:
print(f"train size :{validation}")
print(f" validation size :{split - validation}")
print(f" test size :{len(dataset)-validation}")

In [ ]:
np.random.shuffle(indices)

In [ ]:
train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

batch_size = 64

train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=test_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=validation_sampler
)


In [ ]:
targets_size = len(dataset.class_to_idx)


In [ ]:
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            # conv1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            # conv2
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            # conv3
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            # conv4
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(50176, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, K),
        )

    def forward(self, X):
        out = self.conv_layers(X)

        # Flatten
        out = out.view(-1, 50176)

        # Fully connected
        out = self.dense_layers(out)

        return out

In [ ]:
dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
device="cpu"

In [ ]:
model = CNN(targets_size)


In [ ]:
model.to(device)


In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary

summary(model, (3, 224, 224))


In [ ]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
optimizer = torch.optim.Adam(model.parameters())





In [ ]:
num_epochs = 5
print_every = 10

# Train the model
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    train_loss = 0
    train_correct = 0
    train_total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data and target tensors to the device (GPU) if available
        data, target = data.to(device), target.to(device)

        # Forward pass
        output = model(data)
        loss = criterion(output, target)

        # Backward pass and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update training loss and accuracy
        train_loss += loss.item() * target.size(0)
        train_correct += (output.argmax(dim=1) == target).sum().item()
        train_total += target.size(0)

        # Print loss after every 'print_every' steps
        if batch_idx % print_every == 0:
            print(
                f'Train Epoch: {epoch} [{batch_idx}/{len(train_loader)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    # Calculate average training loss and accuracy
    train_loss /= train_total
    train_acc = train_correct / train_total

    # Evaluate the model on the validation set
    model.eval()

    valid_loss = 0
    valid_correct = 0
    valid_total = 0

    with torch.no_grad():
        for data, target in validation_loader:
            # Move data and target tensors to the device (GPU) if available
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)
            loss = criterion(output, target)

            # Update validation loss and accuracy
            valid_loss += loss.item() * target.size(0)
            valid_correct += (output.argmax(dim=1) == target).sum().item()
            valid_total += target.size(0)

    # Calculate average validation loss and accuracy
    valid_loss /= valid_total
    valid_acc = valid_correct / valid_total

    print(f'Epoch: {epoch+1}/{num_epochs}\t'
          f'Train Loss: {train_loss:.6f}\t'
          f'Train Acc: {train_acc:.4f}\t'
          f'Valid Loss: {valid_loss:.6f}\t'
          f'Valid Acc: {valid_acc:.4f}')

print('Finished Training')


In [ ]:
torch.save(model.state_dict(), 'plant_disease_model_1.pt')
